In [1]:
!pip install geopandas
!pip install pysal
!pip install pysal==2.0.0
!pip install descartes

  Could not find a version that satisfies the requirement pysal==2.0.0 (from versions: 1.4.1, 1.5.0, 1.6.0, 1.7.0, 1.8.0, 1.9.1, 1.10.0, 1.11.0, 1.11.1, 1.11.2, 1.12.0, 1.13.0, 1.14.0, 1.14.1, 1.14.2, 1.14.3, 1.14.4, 1.14.4.post1, 1.14.4.post2)
No matching distribution found for pysal==2.0.0


In [2]:
!pip install --ignore-installed geopandas

  Using cached https://files.pythonhosted.org/packages/5b/0c/e6c99e561b03482220f00443f610ccf4dce9b50f4b1093d735f93c6fc8c6/geopandas-0.6.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9b/9a/04dcffd4ff9bc064c05f723be71177cfb593d5ce6e4d6185c3ef8b7a4be9/pyproj-2.2.2-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/81/d1/b8e1b089a8ddd6df74be583d70373eac55c725c6197c115efbd3c3e1509f/Shapely-1.6.4.post2-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/db/83/7d4008ffc2988066ff37f6a0bb6d7b60822367dcb36ba5e39aa7801fda54/pandas-0.24.2-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/89/dd/289641061a69bb9b302e9fd52e5532f8345f6b277d8f4c98419fbbef1a14/Fiona-1.8.13-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/6a/c8/1b8513155e4bc68f322e2fbaf90708dd66290a0edaa604ea1675357d18ee/aenum-2.2.3-py2-none-any.whl
  Usin

In [3]:
import geopandas as gp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import pylab
import descartes
import numpy as np

%matplotlib inline

plt.style.use('ggplot')
pylab.rcParams['figure.figsize'] = (40., 32.)

In [4]:
data_path = "./data/borough_heatmap/MPS Borough Level Crime (most recent 24 months).csv"

sexassaults = pd.read_csv(data_path)
sexassaults= sexassaults[sexassaults['MajorText']=='Sexual Offences']
sexassaults.head()

,MajorText,MinorText,LookUp_BoroughName,201712,201801,201802,201803,201804,201805,201806,...,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911
35,Sexual Offences,Other Sexual Offences,Barking and Dagenham,17,17,17,30,28,30,36,...,21,27,23,32,41,36,18,33,40,34
36,Sexual Offences,Rape,Barking and Dagenham,21,24,10,17,19,27,15,...,32,28,27,19,26,23,23,30,34,23
84,Sexual Offences,Other Sexual Offences,Barnet,33,36,28,37,23,37,37,...,32,26,23,37,27,27,34,26,25,32
85,Sexual Offences,Rape,Barnet,19,17,16,23,35,54,18,...,12,19,11,16,15,13,18,14,30,26
129,Sexual Offences,Other Sexual Offences,Bexley,20,24,19,27,23,26,22,...,18,18,18,16,19,26,11,20,25,24


In [5]:
sexassaults=sexassaults.set_index(['LookUp_BoroughName'])
sexassaults['2019']=(sexassaults['201901']+sexassaults['201902']+sexassaults['201903']+sexassaults['201904']
                     +sexassaults['201905']+sexassaults['201906']+sexassaults['201907']+sexassaults['201908']
                     +sexassaults['201909']+sexassaults['201910']+sexassaults['201911'])
sexassaults= sexassaults.drop(columns=['201712', '201802','201801','201802','201803','201804','201805',
                                       '201806','201807','201808','201809','201810','201811','201812',
                                       '201901','201902','201903','201904','201905','201906','201907',
                                       '201908','201909','201910','201911'])
sexassaults.head()

,MajorText,MinorText,2019
LookUp_BoroughName,,,
Barking and Dagenham,Sexual Offences,Other Sexual Offences,331
Barking and Dagenham,Sexual Offences,Rape,286
Barnet,Sexual Offences,Other Sexual Offences,323
Barnet,Sexual Offences,Rape,188
Bexley,Sexual Offences,Other Sexual Offences,218


In [6]:
total = sexassaults.groupby(['LookUp_BoroughName'])['2019'].sum()
sexassaults['Total'] = total
sexassaults.head()

,MajorText,MinorText,2019,Total
LookUp_BoroughName,,,,
Barking and Dagenham,Sexual Offences,Other Sexual Offences,331,617
Barking and Dagenham,Sexual Offences,Rape,286,617
Barnet,Sexual Offences,Other Sexual Offences,323,511
Barnet,Sexual Offences,Rape,188,511
Bexley,Sexual Offences,Other Sexual Offences,218,354


In [7]:
data = "data/borough_heatmap/London_Borough_Excluding_MHW.shp"

mapBoroughs = gp.read_file(data)
mapBoroughs.head()

,NAME,GSS_CODE,HECTARES,NONLD_AREA,ONS_INNER,SUB_2009,SUB_2006,geometry
0,Kingston upon Thames,E09000021,3726.117,0.000,F,None,None,"POLYGON ((516401.600 160201.800, 516407.300 16..."
1,Croydon,E09000008,8649.441,0.000,F,None,None,"POLYGON ((535009.200 159504.700, 535005.500 15..."
2,Bromley,E09000006,15013.487,0.000,F,None,None,"POLYGON ((540373.600 157530.400, 540361.200 15..."
3,Hounslow,E09000018,5658.541,60.755,F,None,None,"POLYGON ((521975.800 178100.000, 521967.700 17..."
4,Ealing,E09000009,5554.428,0.000,F,None,None,"POLYGON ((510253.500 182881.600, 510249.900 18..."


In [8]:
geosexassaults = mapBoroughs.merge(sexassaults, left_on= 'NAME', right_on= 'LookUp_BoroughName', how= 'inner')
geosexassaults

,NAME,GSS_CODE,HECTARES,NONLD_AREA,ONS_INNER,SUB_2009,SUB_2006,geometry,MajorText,MinorText,2019,Total
0,Kingston upon Thames,E09000021,3726.117,0.000,F,None,None,"POLYGON ((516401.600 160201.800, 516407.300 16...",Sexual Offences,Other Sexual Offences,202,338
1,Kingston upon Thames,E09000021,3726.117,0.000,F,None,None,"POLYGON ((516401.600 160201.800, 516407.300 16...",Sexual Offences,Rape,136,338
2,Croydon,E09000008,8649.441,0.000,F,None,None,"POLYGON ((535009.200 159504.700, 535005.500 15...",Sexual Offences,Other Sexual Offences,523,979
3,Croydon,E09000008,8649.441,0.000,F,None,None,"POLYGON ((535009.200 159504.700, 535005.500 15...",Sexual Offences,Rape,456,979
4,Bromley,E09000006,15013.487,0.000,F,None,None,"POLYGON ((540373.600 157530.400, 540361.200 15...",Sexual Offences,Other Sexual Offences,347,511
5,Bromley,E09000006,15013.487,0.000,F,None,None,"POLYGON ((540373.600 157530.400, 540361.200 15...",Sexual Offences,Rape,164,511
6,Hounslow,E09000018,5658.541,60.755,F,None,None,"POLYGON ((521975.800 178100.000, 521967.700 17...",Sexual Offences,Other Sexual Offences,298,491
7,Hounslow,E09000018,5658.541,60.755,F,None,None,"POLYGON ((521975.800 178100.000, 521967.700 17...",Sexual Offences,Rape,193,491
8,Ealing,E09000009,5554.428,0.000,F,None,None,"POLYGON ((510253.500 182881.600, 510249.900 18...",Sexual Offences,Other Sexual Offences,398,666
9,Ealing,E09000009,5554.428,0.000,F,None,None,"POLYGON ((510253.500 182881.600, 510249.900 18...",Sexual Offences,Rape,268,666


In [9]:
original_crs = geosexassaults.crs
target_crs = {'datum':'WGS84', 'no_defs':True, 'proj':'merc'}
projected_geosexassaults = geosexassaults.to_crs(crs=target_crs)
geosexassaults

,NAME,GSS_CODE,HECTARES,NONLD_AREA,ONS_INNER,SUB_2009,SUB_2006,geometry,MajorText,MinorText,2019,Total
0,Kingston upon Thames,E09000021,3726.117,0.000,F,None,None,"POLYGON ((516401.600 160201.800, 516407.300 16...",Sexual Offences,Other Sexual Offences,202,338
1,Kingston upon Thames,E09000021,3726.117,0.000,F,None,None,"POLYGON ((516401.600 160201.800, 516407.300 16...",Sexual Offences,Rape,136,338
2,Croydon,E09000008,8649.441,0.000,F,None,None,"POLYGON ((535009.200 159504.700, 535005.500 15...",Sexual Offences,Other Sexual Offences,523,979
3,Croydon,E09000008,8649.441,0.000,F,None,None,"POLYGON ((535009.200 159504.700, 535005.500 15...",Sexual Offences,Rape,456,979
4,Bromley,E09000006,15013.487,0.000,F,None,None,"POLYGON ((540373.600 157530.400, 540361.200 15...",Sexual Offences,Other Sexual Offences,347,511
5,Bromley,E09000006,15013.487,0.000,F,None,None,"POLYGON ((540373.600 157530.400, 540361.200 15...",Sexual Offences,Rape,164,511
6,Hounslow,E09000018,5658.541,60.755,F,None,None,"POLYGON ((521975.800 178100.000, 521967.700 17...",Sexual Offences,Other Sexual Offences,298,491
7,Hounslow,E09000018,5658.541,60.755,F,None,None,"POLYGON ((521975.800 178100.000, 521967.700 17...",Sexual Offences,Rape,193,491
8,Ealing,E09000009,5554.428,0.000,F,None,None,"POLYGON ((510253.500 182881.600, 510249.900 18...",Sexual Offences,Other Sexual Offences,398,666
9,Ealing,E09000009,5554.428,0.000,F,None,None,"POLYGON ((510253.500 182881.600, 510249.900 18...",Sexual Offences,Rape,268,666


In [ ]:
#Install the geopandas module
!pip install --ignore-installed geopandas

#Uninstall any existing version; forcing Yes
!pip uninstall --yes pysal

#Install newest branch
!pip install pysal==1.14.4

  Using cached https://files.pythonhosted.org/packages/5b/0c/e6c99e561b03482220f00443f610ccf4dce9b50f4b1093d735f93c6fc8c6/geopandas-0.6.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9b/9a/04dcffd4ff9bc064c05f723be71177cfb593d5ce6e4d6185c3ef8b7a4be9/pyproj-2.2.2-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/81/d1/b8e1b089a8ddd6df74be583d70373eac55c725c6197c115efbd3c3e1509f/Shapely-1.6.4.post2-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/db/83/7d4008ffc2988066ff37f6a0bb6d7b60822367dcb36ba5e39aa7801fda54/pandas-0.24.2-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/89/dd/289641061a69bb9b302e9fd52e5532f8345f6b277d8f4c98419fbbef1a14/Fiona-1.8.13-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/6a/c8/1b8513155e4bc68f322e2fbaf90708dd66290a0edaa604ea1675357d18ee/aenum-2.2.3-py2-none-any.whl
  Usin

In [ ]:
# Convenience functions for working with colour ramps and bars
def colorbar_index(ncolors, cmap, labels=None, **kwargs):
    """
    This is a convenience function to stop you making off-by-one errors
    Takes a standard colour ramp, and discretizes it,
    then draws a colour bar with correctly aligned labels
    """
    cmap = cmap_discretize(cmap, ncolors)
    mappable = plt.cm.ScalarMappable(cmap=cmap)
    mappable.set_array([])
    mappable.set_clim(-0.5, ncolors+0.5)
    colorbar = matplotlib.pyplot.colorbar(mappable, **kwargs)
    colorbar.set_ticks(np.linspace(0, ncolors, ncolors))
    colorbar.set_ticklabels(range(ncolors))
    if labels:
        colorbar.set_ticklabels(labels)
    return colorbar

def cmap_discretize(cmap, N):
    """
    Return a discrete colormap from the continuous colormap cmap.

        cmap: colormap instance, eg. cm.jet. 
        N: number of colors.

    Example
        x = resize(arange(100), (5,100))
        djet = cmap_discretize(cm.jet, 5)
        imshow(x, cmap=djet)

    """
    if type(cmap) == str:
        cmap = get_cmap(cmap)
    colors_i = np.concatenate((np.linspace(0, 1., N), (0., 0., 0., 0.)))
    colors_rgba = cmap(colors_i)
    indices = np.linspace(0, 1., N + 1)
    cdict = {}
    for ki, key in enumerate(('red', 'green', 'blue')):
        cdict[key] = [(indices[i], colors_rgba[i - 1, ki], colors_rgba[i, ki]) for i in xrange(N+1)]
    return matplotlib.colors.LinearSegmentedColormap(cmap.name + "_%d" % N, cdict, 1024)

from pysal.esda.mapclassify import Quantiles

# to create and split values for the colorbar
breaks = Quantiles(
    geosexassaults['Total'].values,
    k=5)
bar_labels = ['<=%i'% b for b in breaks.bins]

# select the data to analyze, and set a title
geosexassaults.plot(column='Total', colormap='OrRd', k=5)
plt.title('London sexual offences 2019', \
              fontdict={'fontsize': '50',
                        'fontweight' : '3'})

# create an annotation for the  data source
plt.annotate('Source: London Datastore, 2014',
           xy=(0.1, .08), xycoords='figure fraction',
           horizontalalignment='left', verticalalignment='top',
           fontsize=20, color='#555555')

#remove axis
plt.axis('off')

cmap = plt.get_cmap('OrRd')
colorbar_index(ncolors=5, cmap=cmap, shrink=0.5, labels=bar_labels)
plt.savefig('./data/heat_map/2019map_sexassaults.png',bbox_inches = 'tight')